In [2]:
import tensorflow as tf
import numpy as np

In [3]:
file_name_string="test_csv.csv"

In [4]:
filename_queue = tf.train.string_input_producer([file_name_string])

In [6]:
reader = tf.TextLineReader()
key,value = reader.read(filename_queue)

In [8]:
record_defaults = [[1.0], [1.0], [1.0]]
id1, id2, result = tf.decode_csv(value, record_defaults=record_defaults)

In [11]:
# with tf.Session() as sess:
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)
#     is_second_read=0
#     while True:
#         id1_, id2_, result_, line_key = sess.run([id1, id2, result, key])
#         if is_second_read==0 and line_key==line1_name:
#             is_second_read=1
#         elif is_second_read==1 and line_key==line1_name:
#             break
#     coord.request_stop()
#     coord.join(threads)
#     sess.close()

In [12]:
def make_tld(csv_filename, header_lines, delim, batch_size):
    dataset = tf.data.TextLineDataset(filenames=csv_filename).skip(header_lines)

    def parse_csv(line):
        cols_types = [[]] * num_cols_  # all required
        columns = tf.decode_csv(line, record_defaults=cols_types, field_delim=delim)
        return tf.stack(columns)

    dataset = dataset.map(parse_csv).batch(batch_size)
    return dataset

In [13]:
def make_tsd(csv_filename, header_lines, delim, batch_size):
    with open(csv_filename, "r") as f:
        lines = f.readlines()

    data_shape = (len(lines) - header_lines, len(lines[header_lines].strip().split(delim)))
    data = np.empty(shape=data_shape, dtype=np.float32)

    for idx, line in enumerate(lines[header_lines:]):
        columns = [float(el) for el in line.strip().split(delim)]
        data[idx, :] = np.array(columns)

    dataset = tf.data.Dataset.from_tensor_slices(data).batch(batch_size)
    return dataset

In [ ]:
batch_size_ = 100

tld_start = datetime.datetime.now()
    tld = make_tld(csv_filename_, header_lines_, delim_, batch_size_)
    tld_next = tld.make_one_shot_iterator().get_next()
    with tf.Session() as tld_sess:
        tld_sess.run(tf.global_variables_initializer())
        try:
            while True:
                tld_out = tld_sess.run(tld_next)
        except tf.errors.OutOfRangeError:
            print("Done")
    tld_end = datetime.datetime.now()
    print("TextLineDataset: " + str(tld_end - tld_start))

    tsd_start = datetime.datetime.now()
    tsd = make_tsd(csv_filename_, header_lines_, delim_, batch_size_)
    tsd_next = tsd.make_one_shot_iterator().get_next()
    with tf.Session() as tsd_sess:
        tsd_sess.run(tf.global_variables_initializer())
        try:
            while True:
                tsd_out = tsd_sess.run(tsd_next)
        except tf.errors.OutOfRangeError:
            print("Done")
    tsd_end = datetime.datetime.now()
    print("TensorSliceDataset: " + str(tsd_end - tsd_start))